In [1]:
import os
import ast
from dotenv import load_dotenv
import arxiv
from openai import OpenAI

# parse .env file and load all variables
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# initialize open ai client for api calls
client = OpenAI()

## Conversation Memories

#### A- ConversationBufferMemory

In [4]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation:
{history}
Human: {input}
AI:"""

history = ''

def talk(question):

    global history
    
    # put current values of the variables into prompt string
    prompt_after_formatting = prompt.format(input=question,
                                            history=history)

    # make API call
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ]
    )

    # processing open ai's response
    output = completion.choices[0].message.content

    # my way of adding human and ai messages into history
    history += f"Human: {question}\nAI: {output}\n"

    return output
    

In [5]:
talk("Do you know anything about OOP?")
talk("Sorry. What did I ask you?")
talk("What is SQL?")
print(prompt.format(input="current question",history=history))

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation:
Human: Do you know anything about OOP?
AI: Yes, Object-Oriented Programming (OOP) is a programming paradigm that revolves around the concept of objects, which can contain data in the form of fields (attributes or properties) and code in the form of procedures (methods). OOP focuses on organizing code into reusable, modular components, allowing for easier maintenance and scalability. Is there a specific aspect of OOP you'd like more information on?
Human: Sorry. What did I ask you?
AI: You asked me if I know anything about OOP (Object-Oriented Programming).
Human: What is SQL?
AI: SQL stands for Structured Query Language, and it is used to communicate with and manipulate databases. SQL allows users to perform tasks such as retrieving data, creating, updating, and deleting records, and managing a database's structure. It is a 

### B- ConversationSummaryMemory

In [12]:
prompt = """System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
{summary}
Human: {input}
AI:"""

summary_prompt = """System: Your task is to summarize below conversation with emphasis on key points. If nothing given, return ''.
{history}
Summary:"""

history = ''

def talk(question):

    global history

    # put history variable into summary prompt string
    summary_prompt_after_formatting = summary_prompt.format(history=history)

    # make API call for summarization
    summary_completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": summary_prompt_after_formatting}
        ]
    )
    
    # processing open ai's response
    summary = summary_completion.choices[0].message.content

    # put current values of the variables into prompt string
    prompt_after_formatting = prompt.format(input=question,
                                            summary=summary)
    
    # make API call for answering question
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ]
    )

    # processing open ai's response
    output = completion.choices[0].message.content
    print(prompt_after_formatting)
    # my way of adding human and ai messages into history
    history += f"Human: {question}\nAI: {output}\n"

    return output

In [18]:
talk("How are you today?")
talk("Sorry. What did I ask you?")
talk("do you know about football?")
talk("when did england compete in euros first?")
talk("what were the other teams in the same competition?")

System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
The conversation between the human and the AI touched on topics such as the AI's well-being, familiarity with football, England's first participation in the Euros in 1968, and other teams in the same competition. The key points highlighted include the structure of football, England's performance in the 1968 Euros, and the other teams involved in the tournament.
Human: How are you today?
AI:
System: This is a conversation between a software engineer and intellectual AI bot. AI bot is talkative and teaches concepts to the engineer.
Current Conversation Summary:
The conversation involved the AI answering questions about football, specifically about England's first participation in the Euros in 1968 and the other teams in the competition. The AI also mentioned that it is functioning well and ready to assist.
Human: Sorry

'In the 1968 European Championship, besides England, the other teams that competed were Italy, Yugoslavia, and the Soviet Union. Italy emerged as the champions of the tournament after defeating Yugoslavia in the final. If you have any more questions about the competition or football history, feel free to ask!'

## Function-Use (Agents and Tools)

In [2]:
def arxiv_search(args):
    # Construct the default API client.
    client = arxiv.Client()

    # Search for the paper with ID
    search_by_id = arxiv.Search(query=args['query'],
                                id_list=[args['id']])
    # Reuse client to fetch the paper, then print its title.
    first_result = next(client.results(search_by_id))
    
    return first_result.title

def convert_time(args):
    temp = args['time'].split(':')

    return int(temp[0])*3600 + int(temp[1])*60 + int(temp[2])

In [3]:
system_prompt = """Answer the following questions as best you can. You have access to the following tools:

{tools}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Question: {input}
Thought:{agent_scratchpad}"""

tools = {
    'convert_time': 'A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}',
    'arxiv_search': 'A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}'
}

In [4]:
agent_scratchpad = ''

def ask(question):

    global agent_scratchpad

    # convert function description into suitable string for prompt
    tools_str = ''
    for func, desc in tools.items():
        tools_str += f"{func} : {desc}\n"
    
    # put current values of the variables into prompt string
    prompt_after_formatting = system_prompt.format(input=question, 
                                                   agent_scratchpad=agent_scratchpad, 
                                                   tools=tools_str, 
                                                   tool_names=list(tools.keys()))
    print(prompt_after_formatting)

    #  make API call for answering question
    # stop generation when see Observation:
    # as it signs that we need a call a function externally
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_after_formatting}
        ],
        stop=["Observation:"]
    )

    # processing open ai's response
    output = completion.choices[0].message.content
    print(output)
    
    # keep reasoning until see Final Answer: string
    while output.find("Final Answer: ")==-1:

        # append previous reasoning step
        agent_scratchpad += output

        # parse LLM output string into function and its arguments
        function_name = output.split("Action: ")[1].split('\n')[0]
        function_args = ast.literal_eval(output.split("Action Input: ")[1].split('\n')[0])

        # call function externally
        result = globals()[function_name](function_args)

        # append function output to logs
        agent_scratchpad += f"Observation: {result}\n"
        print(f"Observation: {result}\n")
        
        # put current values of the variables into prompt string
        prompt_after_formatting = system_prompt.format(input=question, 
                                                    agent_scratchpad=agent_scratchpad, 
                                                    tools=tools_str, 
                                                    tool_names=list(tools.keys()))
        
        # make the next API call, this time with the result of 
        # previously requested function by LLM 
        completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "user", "content": prompt_after_formatting}
            ],
            stop=["Observation:"]
        )

        # processing open ai's response
        output = completion.choices[0].message.content
        
        print(output)  
    
    return output

In [6]:
ask("how many seconds in 4:00:01. What's the paper 2311.01606 about?")

Answer the following questions as best you can. You have access to the following tools:

convert_time : A function to convert a time string with format H:MM:SS to seconds, args: {"time": {"type": "string"}}
arxiv_search : A function to get information about a scientific article or articles, args: {"query": {"type": "string"}, "id": {"type": "string"}}


ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about what to do. only one action at a time.
Action: the action to take, should be one of ['convert_time', 'arxiv_search']
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Reminder to ALWAYS use the exact characters `Final Answer:` when you provide a definitive answer.

Question: how many seconds in 4:00:01. What's the paper 2

'Final Answer: 14401 seconds. The paper 2311.01606 is about "KG-FRUS: a Novel Graph-based Dataset of 127 Years of US Diplomatic Relations".'